In [25]:
import os
import json
import glob
import numpy as np
import pandas as pd
from astroquery.ipac.ned import Ned
from wiserep_api.properties import get_target_property

## Redshift from Host Galaxy

In [21]:
def load_z_dict():
    """Loads the dictionary with the redshift values.
    """
    redshift_file = 'redshifts.json'
    if os.path.isfile(redshift_file) is True:
        with open(redshift_file, 'r') as fp:
            z_dict = json.load(fp)
    else:
        z_dict = {'name':[], 'z':[], 'source':[], 'host_name':[]}
        
    return z_dict

def save_z_dict(z_dict):
    """Saves the dictionary with the redshift values. 
    """
    with open('redshifts.json', 'w') as outfile:
        json.dump(z_dict, outfile)
        
def get_redshift(target, overwrite=False):
    """Obtains the redshift for the given target.
    """
    z_dict = load_z_dict()
    if target in z_dict['name'] and overwrite is False:
        # quit
        return None
    
    host_name = get_target_property(target, 'host')
    if host_name!='':
        try:
            z_table = Ned.get_table(host_name, table='redshifts')
            z = z_table['Published Redshift'].value[0]
            source = 'host'
        except Exception as exc:
            print(f'{target} ({host_name}): {exc}')
            # quit
            return None
    else:
        z = np.nan
        source = ''

    if target not in z_dict['name']:
        z_dict['name'].append(target)
        z_dict['z'].append(z)
        z_dict['source'].append(source)
        z_dict['host_name'].append(host_name)

    elif target in z_dict['name'] and overwrite is True:
        target_id = z_dict['name'].index(target)

        z_dict['z'][target_id] = z
        z_dict['source'][target_id] = source
        z_dict['host_name'][target_id] = host_name
        
    save_z_dict(z_dict)

In [22]:
peak_spectra = glob.glob('peak_spectra/*')
targets = []
for file in peak_spectra:
    target = os.path.basename(file).split('_')[0]
    if target not in targets:
        targets.append(target)
        
len(targets)

1890

In [23]:
for target in targets:
    get_redshift(target)

2004as (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2006bt (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2003ek (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2008bc (A093831-6358): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpreter.
2008bf (NGC 4055): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2007qe (Anon.): The remote service returned the following error message.
ERROR: The object name that you have submitted is not currently recognized by the NED name interpret

KeyboardInterrupt: 

In [26]:
z_dict = load_z_dict()
z_df = pd.DataFrame(z_dict)
z_df.to_csv('redshifts.csv', index=False)

In [29]:
z_df.dropna()

,name,z,source,host_name
1,1998aq,0.003710,host,NGC3982
2,2004ef,0.030988,host,UGC 12158
3,2001da,0.017381,host,NGC 7780
4,2004S,0.009300,host,M-05-16-21
6,1997Y,0.015952,host,NGC 4675
...,...,...,...,...
404,1995ap,0.300000,host,[K95] J031228.13+004143.4
406,2002dk,0.018533,host,NGC 6616
409,2007ci,0.017952,host,NGC 3873
412,1996ai,0.003156,host,NGC 5005


## Redshift from SNID Templates

In [30]:
directories = glob.glob('spectra/*')
print(f'{len(directories)} SNe')

n_templates = 10
max_phase = +5

8659 SNe


In [ ]:
snid_dict = {'name':[], 'snid_z':[], 'snid_z_std':[]}

for directory in directories:
    sn_name = os.path.basename(directory)
    target_files = glob.glob(os.path.join(directory, '*'))
    # SNID outputs
    snid_files = [file for file in target_files if file.endswith('.output')]

    # iterate over all spectra for this SN
    z_list = []
    z_err_list = []
    for templ_file in snid_files:
        df = pd.read_csv(templ_file, nrows=n_templates)
        mean_phase = df.age.mean()

        if np.abs(mean_phase) <= max_phase:
            # this spectrum is at peak
            z_list = z_list + list(df.z.values)
            z_err_list = z_err_list + list(df.zerr.values)
       
    if len(z_list) == 0:
        snid_z = snid_z_std = np.nan
    else:
        # weighted average taking the SNID z and zerr
        snid_z = np.average(np.array(z_list), 
                            weights=1/np.array(z_err_list)**2)
        snid_z_std = np.std(np.array(z_list))
    
    snid_dict['name'].append(sn_name)
    snid_dict['snid_z'].append(np.round(snid_z, 4))
    snid_dict['snid_z_std'].append(np.round(snid_z_std, 4))
    
snid_z_df = pd.DataFrame(snid_dict)
snid_z_df.dropna(inplace=True)
snid_z_df.to_csv('snid_redshifts.csv', index=False)

In [53]:
master_df = pd.merge(snid_z_df, z_df, how='outer', on='name')
master_df.to_csv('all_redshifts.csv', index=False)
master_df

,name,snid_z,snid_z_std,z,source,host_name
0,1997dt,0.0048,0.0036,0.007318,host,NGC 7448
1,2023gfx,0.0429,0.0039,NaN,NaN,NaN
2,PTF11kjl,0.1251,0.0070,NaN,NaN,NaN
3,iPTF13ayw,0.0522,0.0033,NaN,NaN,NaN
4,2022adwc,0.1216,0.0024,NaN,NaN,NaN
...,...,...,...,...,...,...
1882,1994D,NaN,NaN,0.002058,host,NGC4526
1883,LSQ12hnr,NaN,NaN,NaN,,
1884,ASASSN-14lp,NaN,NaN,NaN,,
1885,iPTF13dge,NaN,NaN,NaN,,
